#### RAG avec la chaine et LCEL (LangChain Expression Language) et une combinaison de sparse et lexical retrievers + reranker
Après avoir expérimenté une chaine de RAG basique dans les notebook v2, j'essaie dans cette v3 les chaînes:


### Librairies

In [ ]:
# ! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain pypdf

Variables d'nvironement 

In [1]:
import dotenv

dotenv.load_dotenv("/home/chougar/Documents/GitHub/Formation_datascientest/DL-NLP/.env")

True

## Import librairies

In [2]:
# libraries and models setup
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.retrievers import SelfQueryRetriever
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_ollama.llms import OllamaLLM
from time import time as timing


#### LLM & modele d'embedding à utiliser


In [3]:
# model_qa_name="qwen2.5:3b"
# model_qa_name="mistral:latest"
# model_qa_name="deepseek-r1:8b"
#model_qa_name="hf.co/bartowski/Falcon3-7B-Instruct-GGUF:Q4_0"
# model_qa_alias="falcon3-7b-mamba"
# llm_qa = OllamaLLM(model=model_qa_name, temperature=0.2)

model_qa_name="gpt-4o-mini"
model_qa_alias=model_qa_name
llm_qa = ChatOpenAI(model_name=model_qa_name, temperature=0.2)



# Embeddings model definition
# model_emb_name='text-embedding-ada-002'
model_emb_name="text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=model_emb_name)


#### Evaluateurs
1. LLM

In [4]:
llm_evaluator = ChatOpenAI(model_name="gpt-4o", temperature=0.2)

2. Modèle spécialisé:
https://huggingface.co/lytang/MiniCheck-Flan-T5-Large

In [5]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU

True
1
NVIDIA GeForce GTX 1080


In [6]:
from minicheck.minicheck import MiniCheck
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "cuda"

loaded_models={"specialized_evaluator": False}

doc = "A group of students gather in the school library to study for their upcoming final exams."
claim_1 = "The students are preparing for an examination."
claim_2 = "The students are on vacation."

# model_name can be one of ['roberta-large', 'deberta-v3-large', 'flan-t5-large', 'Bespoke-MiniCheck-7B']
specialized_evaluator = MiniCheck(model_name='flan-t5-large', cache_dir='./ckpts')
loaded_models["specialized_evaluator"]=True

pred_label, raw_prob, _, _ = specialized_evaluator.score(docs=[doc, doc], claims=[claim_1, claim_2])
# pred_label, raw_prob, _, _ = scorer.score(docs=[doc], claims=[claim_2])

print(pred_label) # [1, 0]
print(raw_prob)   # [0.9805923700332642, 0.007121307775378227]


2025-03-08 19:43:53.694890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741459433.719725   51112 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741459433.727337   51112 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Evaluating: 100%|██████████| 2/2 [00:00<00:00,  3.58it/s]

[1, 0]
[0.9805923700332642, 0.007121328264474869]


#### Step 2: charger le PP

In [7]:
pdf_file_path = './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf'

# Create a loader instance
loader = PyPDFLoader(pdf_file_path)

# Load the data from the PDF
data = loader.load()

#### Step 3: Split  PDF en fragments
Basé sur l'expérience du notebook `rag-v2-gridSearch`, le meilleur modèle et chunk size/overlap sont `text-embedding-ada-002` et 2000 / 400


In [8]:
x=2000
text_splitter = RecursiveCharacterTextSplitter(chunk_size=x, chunk_overlap=x*0.2)

split_docs = text_splitter.split_documents(data)

In [9]:
# nettoyage
bruits=["Planète Urgence | FOREST Programme"]
for doc in split_docs:
    for bruit in bruits:
        if bruit in doc.page_content:
            doc.page_content=doc.page_content.replace(bruit, "")
# del empty docs
split_docs = [doc for doc in split_docs if len(doc.page_content)>0]

#### Step 4: Creation d'un BD Chroma

In [10]:
db_path = "./"

chroma_db = Chroma.from_documents(split_docs, embedding_model,)# persist_directory=db_path,)

faiss_db = FAISS.from_documents(split_docs, embedding_model,)



#### Step 5: Retrait documents pertinents (phase RETRIEVAL)
Basé sur l'expérience du notebook `rag-v2-gridSearch`, le meilleur k=12 et la meilleure méthode de recherche est `mmr`

In [11]:
# Creation des retrievers
#retriever = chroma_db.as_retriever(k=12, search_type="mmr")

search_type="mmr"
retriever_chroma = chroma_db.as_retriever(
    search_kwargs={
        "k": 10, 
        "fetch_k": 20
    },
    search_type= search_type
)

retriever_faiss = faiss_db.as_retriever(
    search_kwargs={
        "k": 10, 
        "fetch_k": 20
    },
    search_type= search_type
)


# Test
query = "Context, environment, project rationale and challenges"

t=timing()
result = retriever_chroma.invoke(query)
print(f"t1 :{timing()-t}")
print(f"chroma_db:")
display(result)

t=timing()
result = retriever_faiss.invoke(query)
print(f"t2 :{timing()-t}")
print(f"faiss:")
display(result)

t1 :0.41508054733276367
chroma_db:


[Document(metadata={'page': 17, 'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf'}, page_content='\nAnother issue that has also had a major impact on life in Indonesia, including\nEast Kalimantan, is the Covid 19 global pandemic that has attacked the\nentire world since early 2020.  The  Covid 19 pandemic has had a major\nimpact on life in Indonesia. East Kalimantan is a province outside Java Island\nwith  the  highest  rate  of  exposure  to  Covid,  which  has  resulted  in  the\ngovernment imposing a lockdown and restrictions on community activities.\nAt the beginning of 2022, community activities began to return to normal,\nbut a new variant emerged, namely Omicron, which spread very quickly.\nFacing a pandemic situation that has not ended, of course, the project must\ncontinue  but  still  pay  attention  to  security,  safety,  and  practice  health\nprotocols.\n3. Strategy & theory of change\nThe  three  years  project  aims  to  contribute  to  restore  the  degr

t2 :0.3517725467681885
faiss:


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 17}, page_content='\nAnother issue that has also had a major impact on life in Indonesia, including\nEast Kalimantan, is the Covid 19 global pandemic that has attacked the\nentire world since early 2020.  The  Covid 19 pandemic has had a major\nimpact on life in Indonesia. East Kalimantan is a province outside Java Island\nwith  the  highest  rate  of  exposure  to  Covid,  which  has  resulted  in  the\ngovernment imposing a lockdown and restrictions on community activities.\nAt the beginning of 2022, community activities began to return to normal,\nbut a new variant emerged, namely Omicron, which spread very quickly.\nFacing a pandemic situation that has not ended, of course, the project must\ncontinue  but  still  pay  attention  to  security,  safety,  and  practice  health\nprotocols.\n3. Strategy & theory of change\nThe  three  years  project  aims  to  contribute  to  restore  the  degr

Sparse retrievers (basé sur la fréquence des mots)


In [12]:
from langchain.retrievers import BM25Retriever, TFIDFRetriever
from sentence_transformers import CrossEncoder

sparse_retriever_tfidf = TFIDFRetriever.from_documents(
    documents=split_docs,
    k=10,
    tfidf_params={"min_df":1,
            "max_df":2,
            "ngram_range":(1, 2)}
    )

sparse_retriever_bm25 = BM25Retriever.from_documents(
    documents=split_docs,
    k=10,
    tfidf_params={"min_df":1,
            "max_df":2,
            "ngram_range":(1, 2)}
    )

# question="Context, environment, project rationale and challenges"
# question='What are the key contextual factors, environmental conditions, project rationale, and potential challenges that may impact the success of the project, including aspects related to biodiversity, climate change impacts, socio-economic context, and health considerations?'
question="Number of People Concerned"
t=timing()
results=sparse_retriever_tfidf.invoke(question)
print(f"time tfidf:", timing()-t)
display(results)

print("\n--------\n")

t=timing()
results=sparse_retriever_bm25.invoke(question)
print(f"time BM25:", timing()-t)
display(results)

time tfidf: 0.003677845001220703


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 40}, page_content="\nlocal  communities  and  the  public  stakeholders  concerned  (villages  and\ndistricts) in order to replicate these activities with the other villages of the\nMahakam Delta.\nThe tree protection from palm tree leaves and net that successfully works to\nprotect tree in previous project will be documented and spread to the key\nrestoration actors for possible replication in their project. In addition, PU\nIndonesia will manage and distribute the publication and knowledge from the\nproject  to  other  parties.  Module  and  material  environmental  education\ncampaign will be used in the scale up project or other locations.\nTable 3. Summary of our lessons learned, and best practices developed on\nprevious projects in the Mahakam Delta\n<table>\nActivities Best practices\n(+)\nLessons learned\n(-)\nChange maker in\nthis new project \nRaising awareness\nin schools \n1/Raisin


--------

time BM25: 0.0020036697387695312


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 49}, page_content='\nOutput 2: Rehabilitate \ndegraded mangrove \nforest\nNumber of Study on land \nsuitability for mangrove \nreforestation\nDocumentation \nactivity\nActivity Report \nReport Document\n1 2022 1 N/A N/A 1\nNumber of Biodiverity \nresearch   to discover \nlinkage beetween habitat \nrestoration and existing of \nendangered species\nDocumentation \nactivity\nActivity Report \nReport Document\n1 2021 1 N/A 1 2\nNumber of nurseries \nEstablished\nDocumentation \nactivity\nActivity Report\nNurseries \ndatabase \nmonitoring \n3 2022 5 6 6 6\nNumber of local \ncommunities involved in \nmangrove nurseries \nDocumentation \nactivity\nActivity Report \nList of \nparticipants\n100 2022 100 100 100 300\nNumber of mangroves \nplanted\nDocumentation \nactivity\nActivity Report\nDatabase \nplanting \nmonitoring\nMap planting \ndistribution\n150.000   2022 230.000    325.00\n0   \n 250.0\n00  

Lexical retriever (basé sur le key word matching)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


import string

def remove_punctuation(text):
    text_no_punctuation = text.translate(str.maketrans('', '', string.punctuation))
    words = text_no_punctuation.split()
    return words

def keyword_association_retriever(query, top_k=10, documents=split_docs, print_keywords=False):
    query_keywords = llm_extract_keyworks.invoke({"question":query})
    # print(query_keywords)
    query_keywords = remove_punctuation(query_keywords)  # Liste des mots-clés dans la requête
    if print_keywords:
        print(query_keywords)
    results = []


    for doc in documents:
        doc_text = doc.page_content.lower()
        # Compter les cooccurrences des mots-clés dans le document
        keyword_matches = sum(1 for word in query_keywords if word.lower() in doc_text)
        # Pondérer le score par la densité des mots-clés
        try:
            score = keyword_matches / len(doc_text.split())
        except Exception as e:
            print(e)
            score=0

        results.append((doc, score))

    # Trier les résultats par score
    ranked_results = sorted(results, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked_results[:top_k]]

system_prompt_exact="""Extract the most relevant keywords from the following text to facilitate searching for important documents containing these terms. 
    Only provide the extracted keywords, without additional text or explanations.
"""

system_prompt_synonyms="""Extract the most relevant keywords from the following text to facilitate searching for important documents.  

- For each extracted keyword, also include its closest synonyms.  
- Return all keywords and synonyms in a **single flat list**, without categorization, explanations, or additional text.  
- Ensure the extracted terms are meaningful and relevant to the text's context.  

### **Output Format:**  
A **comma-separated** flat list of keywords and their closest synonyms.  
"""

prompt_extract_keyworks = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_exact),
        ("human", "{question}"),
    ]
)
llm_extract_keyworks = prompt_extract_keyworks | llm_qa | StrOutputParser()
resp=llm_extract_keyworks.invoke(question)
remove_punctuation(resp)

print(f"Question: {question}")
keyword_association_retriever(query=question, top_k=10, print_keywords=True)

Question: Number of People Concerned
['Number', 'People', 'Concerned']


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 18}, page_content='\nFeature 3. Theory of Change\n4. Beneficiaries\nNumber of direct beneficiaries of the pilot project: 3245 people with the\nproportion of 30% women and 70% men. \nNumber of indirect beneficiaries: 3000 people by assuming at least the\nproject will give benefit indirectly to 1500 people per location including in\nEast Kalimantan and Indonesia.\nThe target groups include: \n- School children (primary schools and secondary schools)\n- Teachers (primary school teachers)'),
 Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 38}, page_content='\ndecide to contribute on the project. By doing this, the project aims to place \nthe local people not as object the project, but the subject of project since \nthey also contribute to design the activities of the project. \nMoreover, regarding the new capital, the project expects to bring the voice

Reranker

In [14]:
# reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# print(reranker.predict(("What is the budget", "The budget is 814.900 €")))

# print(reranker.predict(("What is the budget", "Number of people concerned")))

# QA model
import torch
from transformers import pipeline
# del pipe_candidate1
torch.cuda.empty_cache()
# pipe_question_document_relevance_deepset = pipeline("question-answering", model="deepset/roberta-base-squad2", device=0)
# Use a pipeline as a high-level helper

# pipe_candidate1 = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad", device=0)
pipe_candidate2 = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad", device=0)
# pipe_candidate3 = pipeline("question-answering", model="deepset/roberta-large-squad2", device=0)
# pipe_candidate4 = pipeline("question-answering", model="deepset/xlm-roberta-large-squad2", device=0)




Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [15]:
question = "What is the specific country and city where the project is being implemented?"
# question='what is the budget of this project'
document = "The location of the project is in Paser District (Adang Bay village) and Kutai Kartanegara district (Delta Mahakam) East Kalimantan Province."

# display(pipe_question_document_relevance_deepset(context= document, question=question))
# display(pipe_candidate1(context= document, question=question))
display(pipe_candidate2(context= document, question=question))
# display(pipe_candidate3(context= document, question=question))
# display(pipe_candidate4(context= document, question=question))


{'score': 0.6023038625717163,
 'start': 116,
 'end': 140,
 'answer': 'East Kalimantan Province'}

Hybrid search (combinaison du sémantique, sparse et lexical)

In [16]:
from langchain_core.runnables import chain


dense_retriever = retriever_faiss
sparse_retriever = sparse_retriever_tfidf

@chain
def hybrid_retrieval(query, top_k=8, dense_retriever=dense_retriever, sparse_retriever=sparse_retriever):
    # query = query["question"]
    dense_results = dense_retriever.get_relevant_documents(query)
    sparse_results = sparse_retriever.get_relevant_documents(query)

    # Fusionner
    combined_results = dense_results + sparse_results + keyword_association_retriever(query)
    

    # Supprimer les doublons
    unique_results = {doc.page_content: doc for doc in combined_results}.values()

    
    # Reranking with crossencoder
    # reranked_results = reranker.predict([(query, doc.page_content) for doc in unique_results])
    # sorted_results = sorted(zip(unique_results, reranked_results), key=lambda x: x[1], reverse=True)
    print(f"nb resultats uniques avant reranking: {len(unique_results)}")
    # Reranking with Q/A model
    reranked_results=[{"score": pipe_candidate2(query, doc.page_content)["score"], "doc": doc} for doc in unique_results]
    
    # Filtrer sur 1ere moitiée
    n=min(len(reranked_results), top_k)
    sorted_results=sorted(reranked_results, key=lambda x: x["score"]>0.25, reverse=True)[:n]
    # ajout du score pour analyse
    res=[]
    for doc in sorted_results:
        doc["doc"].metadata["score"]=doc["score"]
        res.append(doc["doc"])
    sorted_results=res    
    
    print(f"nb resultats uniques après reranking: {len(sorted_results)}")
    


    # return [doc for doc, _ in sorted_results[:top_k]]
    return sorted_results

In [17]:
# Eval ran reply with llm
def score_reference_vs_rag_with_gpt(question, reference_text, ragReply, llm=llm_evaluator):
    # prompt=f"""
    #     Help me to compare a RAG answer against a reference text to a given question

    #     Question:\n{question}

    #     Reference text:\n{reference_text} 

    #     RAG anwser:\n{ragReply}

    #     Provide a score for 1 to 10 to evaluate the quality of the RAG answer against the reference text (coherence, coverage, clarity ..)
    #     When the RAG answer states it does have enough information to reply entierly to the question, don't penalize it in your evaluation.
    #     Respond only with the score (1,2 ... 10)
    # """

    prompt=f"""
        Evaluate the quality of a Retrieval-Augmented Generation (RAG) answer by comparing it against a reference text for a given question.  

        ### Inputs:  
        - **Question:** {question}  
        - **Reference Text:** {reference_text}  
        - **RAG Answer:** {ragReply}  

        ### Evaluation Criteria:  
        Rate the RAG answer on a scale of **1 to 10** based on the following aspects:  
        1. **Coherence:** How logically structured and consistent the answer is.  
        2. **Coverage:** How well the answer addresses key points found in the reference text.  
        3. **Clarity:** How clearly the answer conveys information.  

        **Important Consideration:**  
        - If the RAG answer explicitly states that it lacks sufficient information to fully answer the question, do **not** penalize it.  

        ### Output Format:  
        Respond **only** with a single number between **1 and 10** (e.g., 1, 2, ..., 10).  

    """

    resp=llm.invoke(prompt)    
    
    
    try:
        return int(resp.content)
    except:
        return resp.content





In [18]:
# La 'reference_answer' est copiée manuellement du document pour être utilisée comme texte de référence lors de l'évaluation

questions=[
    {
        "q_#": 1,
        "question": "Description of the project", 
        "reference_answer": """    
            Brief project description
            East Kalimantan Province in 2021/2022 received great attention nationally
            because of the moving of the state capital city (Jakarta) to a location near
            the city of Balikpapan and Penajam Paser Utara in East Kalimantan Province.
            The development of the new capital will start in 2022. Although the
            Indonesia President commit to develop the new capital as Forest and Smart
            City, the surrounding area particularly the coastal area such as Delta
            Mahakam and Adang Bay might get high pressure as the consequence of the
            new development and the movement of 1.5 million people to the new
            capital.
            Delta Mahakam, in the eastern part of East Kalimantan, is an area that is
            relatively close to the prospective center of the State capital (about 100 km).
            Mahakam Delta is naturally a mangrove habitat, but due to excessive land
            clearing for extensive aquaculture about 47.5 % of the mangrove ecosystem
            is degraded to be converted into aquaculture (2017). Despite various
            conservation efforts by different parties and the government, land clearing
            still continues. Delta Mahakam land ownership is government land that has
            designated as a production forest, but this area has been inhabited by
            residents from generation to generation.
            Adang Bay is one of the coastal villages in Adang Bay, Paser Regency, on the
            southern part of East Kalimantan Province (about 100 km from the new
            capital). This area is also experiencing land conversion to increase
            aquaculture, besides there are several locations in coastal areas that are
            affected by abrasion. Restoration activities in East Kalimantan Province are
            needed to restore a degraded environment, as well as to support the vision
            of the nation's capital as a green city.
            The ecosystem in Delta Mahakam and Adang Bay 1 are also home to
            critically endangered species, such as the nasal monkey (proboscis
            monkey), endemic to the island of Borneo. On a global scale, the mangrove
            is a key ecosystem to answer the challenge of carbon sequestration and
            fight against climate change.
            The objective of the project is therefore to contribute to restore the
            degraded mangrove forest in East Kalimantan (Delta Mahakam and Adang
            Bay) as home of endemic and endangered species including proboscis
            monkey and key ecosystem to mitigate and to adapt the impact of climate
            change; and this, through four main actions: raising awareness of the
            stakeholders, rehabilitating degraded mangrove forest, supporting the
        """
    },
    {
        "q_#": 2,
        "question": "Country and city", 
        "reference_answer": """
            The location of the project is in Paser District (Adang Bay village) and Kutai
            Kartanegara district (Delta Mahakam) East Kalimantan Province. The location
            of project is nearby the new capital of Indonesia which is in the Penajam
            Paser Utara (around 130-160 km)
        """
    },
    {
        "q_#": 3,
        "question": "Target beneficiaries", 
        "reference_answer": """
            Number of direct beneficiaries of the pilot project: 3245 people with the
            proportion of 30% women and 70% men.
            Number of indirect beneficiaries: 3000 people by assuming at least the
            project will give benefit indirectly to 1500 people per location including in
            East Kalimantan and Indonesia.
            The target groups include:
            - School children (primary schools and secondary schools)
            - Teachers (primary school teachers)
            - Community members (villagers, consists of fish farmers, women group,
            and youth)
            - Village officials
            - Stakeholders from various institutions (government institutions,
            universities, and non-government organizations)
            - Public audience in general (reached by Media)
            Other potential groups:
            - High school and university students
            - Environmental activists 
        """
    },
    {
        "q_#": 4,
        "question": "Number of people concerned", 
        "reference_answer":"""
            Number of direct beneficiaries of the pilot project: 3245 people with the
            proportion of 30% women and 70% men.
            Number of indirect beneficiaries: 3000 people by assuming at least the
            project will give benefit indirectly to 1500 people per location including in
            East Kalimantan and Indonesia.
        """ 
    },
    {
        "q_#": 5,
        "question": "Context, environment, project rationale and challenges", 
        "reference_answer": """
            Context & environment and development challenges
            Geographic and socio-economic context
            East Kalimantan is one of the richest provinces in Indonesia and the main
            contribution to the national GDP. Before palm oil and mining coal booming in
            early 2000, forestry, mining and gas sectors are the backbone of economic
            development in East Kalimantan. Because too much depending on the
            unrenewable natural resources, the economic growth of East Kalimantan
            gradually declines and, in 2016, reached the minus point because of the
            lowest price of coal at the global level. In 2019, Indonesia government has
            decided to move the capital of Indonesia from Jakarta to East Kalimantan.
            Currently, the government accelerate the infrastructure development of new
            capital.

            The project will be implemented in several regions of Mahakam Delta and
            Adang Bay. Mahakam Delta is located on the eastern coast of the island of
            Borneo, in East Kalimantan province, which is one of the five provinces that
            has the lowest population density in Indonesia. This province is also the main
            contributor to the national GDP, mainly for its wealth in oil and gas. It is
            nevertheless aquaculture activities which constitute the main source of
            income for the local population. About 90% of the population depend on it
            for their livelihood. As a result, 54.19% of the Mahakam Delta has been
            converted to shrimp ponds. The majority of exports from the area are made
            up of tiger shrimp and white shrimp that are farmed in the delta ponds and
            along the Paser District's shore.

            Paser District is located on the east coast of East Kalimantan Province. The
            village of Adang Bay is located in the coastal area of this district, in Adang
            Bay. This area is a conservation area managed by the Ministry of Forestry
            (KPHP). Therefore, limited economic activities are allowed in this area.
            However, since the late 1990s, massive clearing for the construction of
            aquaculture ponds has destroyed the mangrove forest in the area. According    
            to the District Pastor's Investment Agency, about 1,506 people live in Adang
            Bay. Most of them work as fishermen, fish farmers and swallowers. The
            village government and the community have made a strong commitment to
            conserve the area by adhering to the jurisdictional REDD+ approach, funded
            by the World Bank's FCPF or Forest Carbon Partnership Facility Project.
            Environmental context
            Largest archipelago in the world (more than 13,000 islands), Indonesia has
            an area of 1,905,000 km2 of which less than 50% is still covered by forests
            today, while the country is part of the 3rd largest planetary tropical
            forest zone (after the Amazon and the Congo Basin). More than half of
            Indonesian forests have disappeared since 1960. However, they are home to
            a large part of the world's biodiversity (more than 10% respectively of plant,
            mammal, reptile and bird species). Today, the country counts for 3 to 5% of
            annual global greenhouse gas emissions (among the 10 most emitting
            countries) including more than 50% due to land use, their change of land use
            and the exploitation of forests.
            Indonesia is home to almost 1/4 of the world's mangroves (20%). This
            maritime ecosystem, made up of a set of mainly woody plants (the most
            notable species being the mangrove), develops in the swinging area of the
            tides of the low coasts and in marshes at the mouth of certain rivers. Of the
            nearly 3.2 million hectares of mangrove forest 2 in the country today, more
            than 50,000 ha are lost each year.
            The mangrove is one of the most productive ecosystems on the planet,
            home to a particularly abundant biomass. The mangroves' root system is
            notably a biotope where a variety of fish and crabs live and reproduce. The
            mangrove thus provides important resources (forestry and fishery) to coastal
            populations, a natural “buffer” zone adapted to salinity, filtering sediment
            and pollution carried by rivers and the sea, and preserving the fresh water
            resources of the land. They are a food security and livelihood issue, in
            particular providing income to fishing communities. This ecosystem is also
            an important natural fount of carbon, with Indonesian mangroves storing
            around 5 times more carbon per hectare than terrestrial forests. The
            government of Indonesia has taken into account this ecosystem in its REDD+
            strategy, implemented in the only pilot province of East-Kalimantan, with the
            Provincial Council on Climate Change (DDPI) with the support of the World
            Bank in the framework of the “Forest Carbon Partnership Facility Project”.
            Finally, the mangrove plays a key role in natural defense. The complex
            network of mangrove roots can help reduce wave power, which limits erosion
            and protects coastal communities from the destructive forces of tropical
            storms. Mangroves provide protection against extreme weather events and
            tsunamis, and can adapt to rising sea levels and subsidence. They therefore    
            contribute to reducing the risk of disasters, to the resilience of communities
            and ecosystems and to their adaptation to climate change.
            In Mahakam Delta, results from a study conducted in 2018 and 2019 by the
            Kutai Kartanegara District has shown that 47.8% of mangrove forests are
            deteriorated.
            Table 1. Critical Criteria of Mahakam Delta Mangrove3
            Critical Criteria Land Area (ha) Percentage
            Damaged 7,034 5.6
            Severe 52,945 42.2
            Undamaged 65,522 52.2
            Total 125,502 100.0
            
            Source: The Result of Spatial Analysis of Mangrove Damage Level (2018)

            With the plan to move the state capital to Penajam Paser Utara District
            (PPU), the development activities to create this new big city will take place
            massively. The central government has planned to create a green city for the
            new capital, which construction will start in 2022, but various problems still
            pose challenges in locations outside the new capital. On the one hand, a
            close government center can control the surrounding environment to keep it
            conserved, but the gap in the quality of human resources and plans to move
            a large number of people from Jakarta to this area will certainly cause
            pressure on the environment.

            Biodiversity issues
            The mangrove of Mahakam Delta conceals a rich marine and arboreal
            biodiversity, characterized by a large variety of fish, arthropods, reptiles
            such as the marine crocodile (Crocodylus porosus), aquatic mammals such
            as the Irrawaddy dolphin (Orcaella brevirostris) or terrestrial like the nasal
            monkey (Nasalis larvatus), these last 2 species being considered as being
            “endangered” by the IUCN.
            The deforestation of Mahakam Delta’s mangrove hampers the effort to
            conserve this type of species, for example by fragmenting the habitat of the
            nasal monkey, whose interaction between populations strongly depends on
            the continuity of the canopy. The isolation of these populations makes them
            more vulnerable to poaching. The long-nosed monkey, endemic of Borneo
            Island is listed as “Endangered” by the IUCN as it has undergone extensive    
            population reductions across its range, and ongoing hunting and habitat
            destruction continue to threaten most populations. Numbers have
            declined by more than 50% (but probably less than 80%) over the past 3
            generations (approximately 36-40 years).4 At the scale of Mahakam Delta,
            only 2 censuses have been conducted to monitor this specie, respectively in
            1997 and 2005 which reflects the lack of resources of local institutions to
            conserve and protect this biodiversity.
            In addition, the degradation of this ecosystem leads to a decrease in fish
            stocks in the delta, threatening both fishermen and species such as sea
            crocodiles and dolphins. The situation is currently might threatening to
            exacerbate human-animal conflicts and therefore to further decrease the
            populations of the above-mentioned species, even threatening them with
            extinction. Hence, beside preventing the mangrove forest conversion into
            palm oil plantation, aquaculture ponds, and other usages, the reforestation
            activity is necessary to improve the degraded mangrove ecosystem in the
            coastal area.
            Paradoxically, the considerable modification of delta habitats resulted in a
            very substantial increase in populations of birds associated with open
            wet areas, such as egrets (100 individuals in 1987 to nearly 15,000
            individuals in 2013). Likewise, some species of heron have seen their
            population sizes increase considerably, such as the purple heron or the Javan
            pond-heron, the lesser adjutant, ducks, Sunda teal and the wandering
            whistling-duck also seem to have used the habitats created by the clearings
            to considerably increase their populations.
            The populations of these species have benefited of new feeding areas when
            the shrimp ponds were developed. Indeed, egrets, ducks, and waders use
            the shrimp ponds in high numbers on cyclical basis when shrimp ponds are
            emptied for shrimp harvesting. The presence of pristine areas, with large
            trees or dense copses of smaller species (Nypa) removed from human
            presence, is also favourable for the reproduction of these species. Here they
            find quiet conditions for reproduction or gatherings (dormitories). Amongst
            the species observed in 2013 and those not observed in 1987, eight dwell in
            an aquatic environment and directly depend on the shrimp ponds: darter,
            stilts, grey heron, black-crowned night heron, intermediate egret, western
            marsh-harrier and the Garganey. The opening of shrimp ponds was the
            obvious factor leading to the growth of all these bird populations.
            
            Institutional Context
            The key players in coastal region in East Kalimantan including in Delta
            Mahakam Ulu (Delta Mahakam) and Delta Mahakam (Adang Bay) are
            relatively similar. Since the area located or nearby the conservation area and
            forest production area, the Ministry of Forestry via Nature Conservancy
            Agency in East Kalimantan and Forest Management Unit (provincial
            government agency) are the most influence actor. They have authority to
            determine the activities which allowed and not allowed in the area. However,
            they cannot control the vast area of conservation area since 50 percent of
            mangrove forest in the region have been degraded. Besides government, the
            others key actors are fishermen, fish farmers, swallow workers and investors
            in aquaculture sectors. Those actors have shaped the landscape of coastal
            area in Delta Mahakam and Adang Bay over the past 20 years. In their hand
            the future of sustainable aquaculture is determined. Environmental and
            development NGOs, oil and gas company and other parties has programme
            in their area. Most of the programme focus on improving the livelihood of the
            local people and restoring the mangrove forest.
            
            The Movement of Indonesian New Capital
            Paser District (East Kalimantan Province) will soon be the site of Indonesia's
            new political capital, Nusantara, as part of the plan to move the country's
            capital from the island of Java to the island of Borneo, which is home to one
            of the world’s largest rainforests.
            Jakarta, the current political capital which will become the country's
            economic capital by 2045, is currently facing several environmental, climatic
            and demographic problems and challenges: overpopulation, heavy pollution,
            rising water levels, frequent flooding, etc. In order to deal with the inevitable
            future security issue, the Indonesian government has decided to build a new
            capital 2,000 km away from Jakarta, in the province of East Kalimantan,
            more precisely between the towns of Balikpapan and Samarinda. With the
            legislation for the relocation of the new capital published, the physical
            development of the new capital will begin in 2022. In August 2024, the
            President plans to celebrate Indonesia's Independence Day in the new
            capital.
            The government plans to make the new capital a "forest city" by strongly
            preserving forest areas and using sustainable energy. However, many argue
            that the development of the new capital could lead to environmental
            degradation and loss of essential biodiversity, especially in the mangrove
            forest. The majority of the Indonesian population, including the local
            population, supports the new capital movement by echoing the effect of
            equitable development. Indeed, for decades, the natural resources of
            Kalimantan Island have been exploited to support Indonesian development,
            especially that of Java Island.
            The location of the project (Delta Mahakam and Adang Bay -Adang Bay) is an
            area relatively close to the potential centre of the state capital (about 100-
            200 km).

            Environment and development challenges
            a) Aquaculture industry
            Mahakam Delta area is under pressure from both the industrial and
            agricultural sectors, including aquaculture facing a national high dynamic.
            From 2015 to 2035 it is expected a destruction of 600,000 ha of mangrove
            for shrimp farm at the national scale. The World Bank (2013) estimates a
            pressure to double cultivated shrimp production from currently 300,000 t
            (produced by 600,000 ha of ponds) to 600,000 t/1,000,000 t by 2030 to fulfil
            the demand. However, with improvements in brackish water aquaculture
            productivity, halting palm oil concession to use mangroves, along with
            maintaining other mangrove use pressures at moderate levels, the net loss
            of mangroves in the next two decades could be reduced to around 23,000 ha
            at this same scale.
            The East-Kalimantan Province is the new area to develop aquaculture ponds
            as Java, Sumatra and Sulawesi islands are facing a decrease of the
            production and the destruction of their environment due to unsustainable
            practices.
            Feature 1: Forecasted mangrove loss at six mangrove regions in Indonesia
            in the next two decades due to land use change under pessimistic scenario.
            Circle size indicates potential loss areas in Sumatra, Kalimantan, and Papua;
            as for Java, Sulawesi and Maluku potential loss areas are represented by the
            smaller circles.
            Scientific studies also show that the percentage of mangrove natural
            recovery is higher in East-Kalimantan with 1.4%/year against 0.7%/year in
            other islands in inactive ponds. This suggests to consider conservation
            activities in specific areas of Mahakam Delta. At the scale of Mahakam Delta,
            the table below for which the percentage (43.7%) is as higher as the
            remaining mangrove forest (48.5%) highlights the dominance of aquaculture.

            
            b) Demography
            The demographic issue must also be considered. Indeed, the announcement
            in 2019 of the relocation of the political and administrative capital of Jakarta
            to the province of East Kalimantan, between the cities of Balikpapan and
            Samarinda, suggests strong migrations, the development of infrastructures
            but also a growing demand for aquaculture products. By 2024, the
            Indonesian Minister of Planning hopes to transfer nearly 1.5 million public
            officials and political representatives in East Kalimantan.
            Delta Mahakam Ulu village, which belongs to Delta Mahakam district, is
            located in the northern part of the Mahakam Delta. The location of Delta
            Mahakam sub-district is close to the state-owned oil company (Pertamina),
            formerly VICO. Due to the proximity of a fairly large company, the
            community's economy is quite dynamic and the area offers a variety of jobs.
            However, the number of people who still carry out the traditional work of
            fishermen and fish farmers is still quite high, especially in the coastal areas.
            Working as a fish farmer has become one of the choices of the community as
            land is available for opening ponds. The conversion of mangrove forests into
            ponds has been going on for decades, but the production of fish and shrimp
            has decreased from time to time. Based on various studies and research,
            planting a number of mangroves in ponds can improve the soil and water
            quality in the ponds so that they can provide sustainable production. The
            farmer groups in Delta Mahakam Ulu ponds are beginning to realise the
            importance of planting mangroves in the ponds, and therefore need support
            from various parties.
            c) Other issues
            The table below represents a summary of estimation of potential loss and
            gain of mangroves in six major regions by 2035. The Kalimantan Island is the
            one to analyze in order to justify Planète Urgence and partners’ information.
            The analysis does not yet consider the movement of new capital issue which
            very likely affect the mangrove forest in East Kalimantan as well.
            
            This table highlight the multiple and complex context in which mangrove loss
            depends and confirms challenges faced in Mahakam Delta area. The lack of
            resources (financial, human resources, material) of local authorities coupled
            with a lack of transparency, coordination and communication around
            responsibilities of each actors impacts the management of mangrove forests,
            natural resources and territorial development.    
            
            Another issue that has also had a major impact on life in Indonesia, including
            East Kalimantan, is the Covid 19 global pandemic that has attacked the
            entire world since early 2020. The Covid 19 pandemic has had a major
            impact on life in Indonesia. East Kalimantan is a province outside Java Island
            with the highest rate of exposure to Covid, which has resulted in the
            government imposing a lockdown and restrictions on community activities.
            At the beginning of 2022, community activities began to return to normal,
            but a new variant emerged, namely Omicron, which spread very quickly.
            Facing a pandemic situation that has not ended, of course, the project must
            continue but still pay attention to security, safety, and practice health
            protocols.
            
            3. Strategy & theory of change
            The three years project aims to contribute to restore the degraded of
            mangrove ecosystem in Production Forest (Mahakamm Delta) and
            Conservation area (Adang Bay). In doing so, the project will address the key
            problems in those regions:
            a. Lack of awareness of local people on mangrove ecosystem,
            biodiversity issue and waste
            b. Huge area of degraded mangrove forest which affect the resilience of
            local people in facing climate change, the habitat of endangered
            species and local economy;
            c. Lack of alternative sustainable livelihood in coastal area;
            d. Poor governance particularly on mangrove ecosystem and its
            environmental and economy issue.
            To overcome those problems, Planet Urgence and its partners will work by
            implementing the PU FORET strategy which rely on three components:
            1. Restore degraded forest;
            2. Environmental awareness;
            3. Strengthening livelihood of local people.
            In addition, the involvement of local NGOs, local community and volunteer is
            key for the successful of the project and the sustainability the impact of the
            project. Therefore, PU will reinforce the capacity of those local stakeholders
            to ensure they can carry out the project activities and together achieve the
            long-term goal of the project.    
            """
    },
    {
        "q_#": 6,
        "question": "Project start date / end date", 
        "reference_answer": "March 2023 – February 2026"
    },
    {
        "q_#": 7,
        "question": """Project budget 
            Total amount of the project (in Euros) 
            
            Amount of donation requested from the Foundation (in Euros) 
            
            Detailed provisional project budget 
            
            Detailed project budget for current year
        """, 
        "reference_answer": "The total required resources is 818 341 € for the period 2023-2026"
    },                

]
 



#### Reformulation de la question initiale

In [19]:

"v1"
# system = """You are a question re-writer that converts an input question to a better version that is optimized \n
#      for vectorstore retrieval. Re-write the input in an interrogative, clear and concise way."""

"v2"
# system="""
#     You are a **question rewriter** that refines input questions to optimize them for **vector store retrieval**.  

#     ### **Instructions:**  
#     - Convert the input into a **clear, concise, and well-structured interrogative sentence**.  
#     - Ensure the reworded question **preserves the original intent** while improving retrieval effectiveness.  
#     - Remove ambiguity and redundant phrasing to enhance search relevance.  

#     Here is the initial question: \n\n {question} \n Formulate an improved question.

#     ### **Optimized Output:**  
#     Provide only the rewritten question, without any additional text.  

# """

"v3"
system="""
    You are a question rewriter tasked with improving input questions to optimize them for vector store retrieval. 
    Your mission is to refine, rephrase, and enhance the provided questions to ensure they are:
    * Clear and easy to understand.
    * Focused.
    * Optimized for effective retrieval by removing ambiguities.
    * Written in an interrogative form while preserving the original intent.

    #### Input Fields to Rework:
    Example 1:
    * Project Description: Reword questions that focus on the project’s overall scope and objectives.
    Example 2:
    * Country and City: Refine questions to specifically inquire about the project’s location.
    Example 3:
    * Target Beneficiaries: Enhance questions to clarify the population or group that benefits from the project.    
    * Number of People Concerned: Rework questions to quantify how many people the project impacts.
    Example 4:
    * Context, Environment, Project Rationale, Challenges, include these topics to enhance the final output:
            * Environmental Context
            * Biodiversity
            * Climate Change impacts
            * Socio-Economic Context
            * Economic, Demographic, Community impacts
            * Institutional context
            * Health context
        Rephrase questions that ask for background information, challenges, and the reasoning behind the project.

    Example 5:
    * Project Start Date / End Date: Rework questions regarding the project’s timeline.
    Example 6:
    * Financial Information:
        * Project Budget: Reword questions about the overall project budget.
        * Total Project Cost: Rephrase inquiries about the total cost of the project.
        * Donation Request Amount: Refine questions asking about the amount of funding requested.
        * Provisional Project Budget: Rework questions about the detailed provisional budget for the project.
        * Current Year Budget: Enhance questions related to the budget specific to the current year.


    #### Response Format:
    For each input question, rephrase it in a clear, and interrogative form, optimized for vector store retrieval. Return only the reworked question.
"""

"v4"
# system = """
#     You are a question rewriter tasked with improving input questions to optimize them for vector store retrieval. 
#     Your mission is to refine, rephrase, and enhance the provided questions to ensure they are:
#     * Clear and easy to understand.
#     * Focused.
#     * Optimized for effective retrieval by removing ambiguities.
#     * Written in an interrogative form while preserving the original intent.
#     * **Expanded with relevant details when applicable** (e.g., including specific topics related to the question).  

#     #### Input Fields to Rework:
#     Example 1:
#     * Project Description: Reword questions that focus on the project’s overall scope and objectives.
#     Example 2:
#     * Country and City: Refine questions to specifically inquire about the project’s location.
#     Example 3:
#     * Target Beneficiaries: Enhance questions to clarify the population or group that benefits from the project.    
#     * Number of People Concerned: Rework questions to quantify how many people the project impacts.
#     Example 4:
#     * Context, Environment, Project Rationale, Challenges: When reformulating questions related to this category, **explicitly incorporate** relevant aspects, such as:
#         - Environmental Context
#         - Biodiversity
#         - Climate Change Impacts
#         - Socio-Economic Context
#         - Economic, Demographic, and Community Impacts
#         - Institutional Context
#         - Health Context  

#     *Ensure that reformulated questions include these aspects where relevant.*

#     Example 5:
#     * Project Start Date / End Date: Rework questions regarding the project’s timeline.
#     Example 6:
#     * Financial Information:
#         * Project Budget: Reword questions about the overall project budget.
#         * Total Project Cost: Rephrase inquiries about the total cost of the project.
#         * Donation Request Amount: Refine questions asking about the amount of funding requested.
#         * Provisional Project Budget: Rework questions about the detailed provisional budget for the project.
#         * Current Year Budget: Enhance questions related to the budget specific to the current year.

#     #### Response Format:
#     For each input question, rephrase it in a **clear, interrogative form** optimized for vector store retrieval.  
#     *If the question relates to a category with specific example topics, explicitly integrate those topics into the reworded question.*  
#     Return **only the reworked question**.
# """

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)
llm_rewriter = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.2)
question_rewriter = re_write_prompt | llm_rewriter | StrOutputParser()
# question="""Context, environment, project rationale and challenges"""
question="Country and City"
# question="Number of people concerned"
# question="Description of the project"
print(f"Initial question:\n{question}")
question_rewriter.invoke({"question": question})

Initial question:
Country and City


'What is the specific country and city where the project is located?'

Methode nouvelle Runnable / LCEL / "chain"

In [20]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda


rag_prompt = ChatPromptTemplate.from_template("""
    Answer the question based **only** on the provided context.  

    - If the context contains enough information to provide a complete or partial answer, use it to formulate a detailed and factual response.  
    - If the context lacks relevant information, respond with: "I don't know."  

    ### **Context:**  
    {context}  

    ### **Question:**  
    {input}  

    ### **Answer:**  
    Provide a clear, factual, and well-structured response based on the available context. Avoid speculation or adding external knowledge.  
""")


lcel_qa_chain = (
    RunnableParallel({
        "context": hybrid_retrieval | (lambda split_docs: "\n\n".join(d.page_content for d in split_docs)),
        "sources": hybrid_retrieval,
        "input": RunnablePassthrough(),
    })
    | (lambda inputs: {
        "answer": (rag_prompt | llm_qa | StrOutputParser()).invoke(inputs),
        "sources": inputs["sources"]  
    })
)

# Usage
question="Number of people concerned"
# question="Context, environment, project rationale and challenges"
# question="Project start date / end date"
# question="Country and city"

reference_answer=[v["reference_answer"] for v in questions if v["question"]==question][0]

enhanced_question=question_rewriter.invoke({"question": question})
print("Question:\n", question)
print("Enhanced_question:\n", enhanced_question)
print("\n----\nReference_answer:\n", reference_answer)
response = lcel_qa_chain.invoke(enhanced_question)
print("\n----\nLCEL_qa_chain answer:\n", response["answer"])
print("\n----\nLCEL_qa_chain sources:\n", response["sources"])



# eval with llm
llm_score= score_reference_vs_rag_with_gpt(enhanced_question, reference_answer, response["answer"])

# eval with specialized model
t=timing()
pred_label, raw_prob, _, _ = specialized_evaluator.score(docs=[reference_answer], claims=[response["answer"]])
print(f"mini check timing: {timing()-t}")
print(f"""LLM score: {llm_score}\n""")
print(f"""specialized model score: {pred_label}, {raw_prob}""")


Question:
 Number of people concerned
Enhanced_question:
 How many individuals are impacted by the project?

----
Reference_answer:
 
            Number of direct beneficiaries of the pilot project: 3245 people with the
            proportion of 30% women and 70% men.
            Number of indirect beneficiaries: 3000 people by assuming at least the
            project will give benefit indirectly to 1500 people per location including in
            East Kalimantan and Indonesia.
        


/tmp/ipykernel_51112/629325432.py:10: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  dense_results = dense_retriever.get_relevant_documents(query)


nb resultats uniques avant reranking: 22


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


nb resultats uniques avant reranking: 22
nb resultats uniques après reranking: 8
nb resultats uniques après reranking: 8


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



----
LCEL_qa_chain answer:
 The project impacts a total of 6,245 individuals. This includes 3,245 direct beneficiaries, which consists of 30% women and 70% men, and 3,000 indirect beneficiaries, who are assumed to benefit indirectly from the project activities.

----
LCEL_qa_chain sources:
 [Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 18, 'score': 0.7263355255126953}, page_content='\nFeature 3. Theory of Change\n4. Beneficiaries\nNumber of direct beneficiaries of the pilot project: 3245 people with the\nproportion of 30% women and 70% men. \nNumber of indirect beneficiaries: 3000 people by assuming at least the\nproject will give benefit indirectly to 1500 people per location including in\nEast Kalimantan and Indonesia.\nThe target groups include: \n- School children (primary schools and secondary schools)\n- Teachers (primary school teachers)'), Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page'

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  6.05it/s]

mini check timing: 0.1704702377319336
LLM score: 8

specialized model score: [0], [0.13304881751537323]


In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

from langchain_core.runnables import chain


dense_retriever = retriever_chroma

@chain
def hybrid_retrieval_base(query, top_k=8):
    query = query["input"]
    dense_results = dense_retriever.get_relevant_documents(query)
    sparse_results = sparse_retriever_tfidf.get_relevant_documents(query)

    # Fusionner
    combined_results = dense_results + sparse_results + keyword_association_retriever(query)

    # Supprimer les doublons
    unique_results = {doc.page_content: doc for doc in combined_results}.values()

    # Reranking
    reranked_results = reranker.predict([(query, doc.page_content) for doc in unique_results])
    sorted_results = sorted(zip(unique_results, reranked_results), key=lambda x: x[1], reverse=True)

    return [doc for doc, _ in sorted_results[:top_k]]
    # return unique_results

# question_answer_chain = create_stuff_documents_chain(llm, rag_prompt)
# chain = create_retrieval_chain(hybrid_retrieval_base, question_answer_chain)

# resp= chain.invoke({"input":"What is the specific country and city where the project is located ?"})
# print(resp["input"])
# print(resp["answer"])
# print(resp["context"])

# llm_score= score_reference_vs_rag_with_gpt(enhanced_question, reference_answer, resp["answer"])
# print(f"""LLM score: {llm_score}\n""")

#### Grille de recherche pour les 2 méthodes
`similarity_score_threshold` étant une version rafinée de similarity, inutile de tester cette dernière

In [23]:
from itertools import product
""" search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7, "k": 8},

    search_type="mmr",
    search_kwargs={"fetch_k": 20, "lambda_mult": 0.5, "k": 8}
"""

sparse_retriever_types=[
    {"type": "sparse", "label": "tfidf", "instance": sparse_retriever_tfidf}, 
    {"type": "sparse", "label": "bm25", "instance": sparse_retriever_bm25,}
]
dense_retriever_types=[
    {"type": "dense", "label": "chroma", "instance": retriever_chroma}, 
    {"type": "dense", "label": "faiss", "instance": retriever_faiss}
]


gridSearch=list(product(dense_retriever_types, sparse_retriever_types,))
gridSearch

[({'type': 'dense',
   'label': 'chroma',
   'instance': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f66f4c12f50>, search_type='mmr', search_kwargs={'k': 10, 'fetch_k': 20})},
  {'type': 'sparse',
   'label': 'tfidf',
   'instance': TFIDFRetriever(vectorizer=TfidfVectorizer(max_df=2, ngram_range=(1, 2)), docs=[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 0}, page_content='MANGROVE RESTORATION AND\nLOCAL DEVELOPMENT PROJECT\nFOREST PROGRAMME\nProject title: MAHAKAM\nCountry: Indonesia\nImplementing  partners:  Yayasan\nMangrove Lestari & Pokja Pesisir'), Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 1}, page_content="\nBrief project description\nEast Kalimantan Province in 2021/2022 received great attention nationally\nbecause of the moving of the state capital city (Jakarta) to a location near\nthe city of 

### Test avec méthode LCEL

In [24]:
def update_lcel_chain(
    dense_retriever, sparse_retriever, keyword_association_retriever=keyword_association_retriever, 
    reranker=pipe_candidate2, llm_qa=llm_qa, rag_prompt=rag_prompt, docs=split_docs,
):
    from langchain_core.runnables import chain
    
    @chain
    def _hybrid_retrieval(
        query, 
        dense_retriever=dense_retriever, 
        sparse_retriever=sparse_retriever, 
        keyword_association_retriever=keyword_association_retriever, 
        reranker=reranker,
        top_k=8,
    ):
        # query = query["question"]
        dense_results = dense_retriever.get_relevant_documents(query)
        sparse_results = sparse_retriever.get_relevant_documents(query)

        # Fusionner
        combined_results = dense_results + sparse_results + keyword_association_retriever(query)

        # Supprimer les doublons
        unique_results = {doc.page_content: doc for doc in combined_results}.values()

        # Reranking with Q/A model
        reranked_results=[{"score": reranker({"question":query, "context":doc.page_content})["score"], "doc": doc} for doc in unique_results]
        print(f"Retrieved unique docs: {len(unique_results)}")
        # Filtrer sur 1ere moitiée
        n=min(len(reranked_results), top_k)
        sorted_results=sorted(reranked_results, key=lambda x: x["score"]>0.25, reverse=True)[:n]
        print(f"Sorted docs: {len(sorted_results)}")
        
        # ajout du score pour analyse
        res=[]
        for doc in sorted_results:
            doc["doc"].metadata["score"]=doc["score"]
            res.append(doc["doc"])
        sorted_results=res    

        return sorted_results

    _lcel_qa_chain = (
        RunnableParallel({
            "context": _hybrid_retrieval | (lambda split_docs: "\n\n".join(d.page_content for d in split_docs)),
            "sources": _hybrid_retrieval,
            "input": RunnablePassthrough(),
        })
        | (lambda inputs: {
            "answer": (rag_prompt | llm_qa | StrOutputParser()).invoke(inputs),
            "sources": inputs["sources"]  
        })
    )


    return _lcel_qa_chain
 


evaluations_lcel=[]
import torch

def retrieval_LCEL_loop(evaluations_lcel=evaluations_lcel):
    for dense_retriever_params, sparse_retriever_params in gridSearch:
        print(f"""Dense retriever type: {dense_retriever_params["label"]}""")
        print(f"""Sparse retriever type: {sparse_retriever_params["label"]}""")
        for q in questions:
            enhanced_question= question_rewriter.invoke({"question": q["question"]})
            questions_variations=[
                {"question": q["question"], "enhanced_question": False},
                {"question": enhanced_question, "enhanced_question": True},
            ]

            for q_v in questions_variations:
                question=q_v["question"]

                print(f"""Question: {question}""")
                print(f"""Question enhanced: {q_v["enhanced_question"]}""")

                retrievalQA_chain=update_lcel_chain(dense_retriever_params["instance"], sparse_retriever_params["instance"])
                rag_qa = retrievalQA_chain.invoke(question)
            
                

                eval_score=score_reference_vs_rag_with_gpt(q["question"], q["reference_answer"], rag_qa["answer"])

                pred_label, raw_prob, _, _ = specialized_evaluator.score(docs=[q['reference_answer']], claims=[rag_qa["answer"]])        
                # score_qa_pipe= pipe_question_document_relevance_deepset(q['reference_answer'], rag_qa["answer"])        
                # score_qa_pipe=score_qa_pipe["score"]
                torch.cuda.empty_cache()
        

                
                print(f"""LLM Eval_score: {eval_score}""")
                print(f"""MiniCheck score: {raw_prob}""")
                # print(f"""Model QA score: {score_qa_pipe}""")

                evaluations_lcel.append(
                    {
                        "question_#":q["q_#"], 
                        "question": question,  
                        "reference_answer": q["reference_answer"],
                        "rag_answer": rag_qa["answer"],
                        "rag_sources": rag_qa["sources"],
                        "eval_score_llm": eval_score,
                        "eval_score_minicheck": raw_prob,
                        # "eval_score_qa_model": score_qa_pipe,
                        "enhanced_question": q_v["enhanced_question"],
                        "dense_retriever_type": dense_retriever_params["label"],
                        "spare_retriever_type": sparse_retriever_params["label"],
                        
                    }
                )
                print("\n-------")
                
            print("\n--------------------")

retrieval_LCEL_loop()

Dense retriever type: chroma
Sparse retriever type: tfidf
Question: Description of the project
Question enhanced: False


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Retrieved unique docs: 23
Sorted docs: 8
Retrieved unique docs: 23
Sorted docs: 8


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]


LLM Eval_score: 6
MiniCheck score: [0.02050635777413845]

-------
Question: What is the detailed description of the project, including its scope and objectives?
Question enhanced: True


...

LLM Eval_score: 8
MiniCheck score: [0.31312671303749084]

-------
Question: What is the total project budget in Euros, including the amount of donation requested from the Foundation, the detailed provisional project budget, and the detailed project budget for the current year?
Question enhanced: True


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Retrieved unique docs: 26
Sorted docs: 8
Retrieved unique docs: 26
Sorted docs: 8


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
Evaluating: 100%|██████████| 1/1 [00:00<00:00,  5.67it/s]

LLM Eval_score: 8
MiniCheck score: [0.18756112456321716]

-------

--------------------


Analyse des résultats

Résultats LCEL

In [25]:
import pandas as pd

enhanced_question=[False, True]
params=list(product(dense_retriever_types, sparse_retriever_types, enhanced_question))

df_evaluations_lecel=pd.DataFrame(evaluations_lcel)
best_params=[]

for dense_params, sparse_params, enhanced_question in params:
    print(f"""Dense type: {dense_params["label"]},\n Sparse type: {sparse_params["label"]},\n enhanced prompt: {enhanced_question}""")
    
    _df=df_evaluations_lecel[(df_evaluations_lecel["dense_retriever_type"]==dense_params["label"])&
        (df_evaluations_lecel["spare_retriever_type"]==sparse_params["label"])&
        (df_evaluations_lecel["enhanced_question"]==enhanced_question)]
    
    eval_score_llm=_df["eval_score_llm"].mean()
    print(f"Score moyen llm eval: {eval_score_llm}")
    if eval_score_llm>=5:
        display(_df)

    best_params.append((dense_params["label"], sparse_params["label"], enhanced_question, eval_score_llm))
    print('\n-----------\n')



Dense type: chroma,
 Sparse type: tfidf,
 enhanced prompt: False
Score moyen llm eval: 7.571428571428571


,question_#,question,reference_answer,rag_answer,rag_sources,eval_score_llm,eval_score_minicheck,enhanced_question,dense_retriever_type,spare_retriever_type
0,1,Description of the project,\n Brief project description\n ...,"The project, implemented by Planète Urgence an...",[page_content='\nHuman resources – Planète Urg...,6,[0.02050635777413845],False,chroma,tfidf
2,2,Country and city,\n The location of the project is i...,"The country is Indonesia, and the relevant cit...",[page_content='MANGROVE RESTORATION AND\nLOCAL...,6,[0.034841276705265045],False,chroma,tfidf
4,3,Target beneficiaries,\n Number of direct beneficiaries o...,The target beneficiaries of the pilot project ...,[page_content='\nFeature 3. Theory of Change\n...,8,[0.12091516703367233],False,chroma,tfidf
6,4,Number of people concerned,\n Number of direct beneficiaries o...,The number of direct beneficiaries of the pilo...,[page_content='\nTable 2. Types of Land Cover ...,9,[0.1603333204984665],False,chroma,tfidf
8,5,"Context, environment, project rationale and ch...",\n Context & environment and develo...,The context outlines the significant impact of...,[page_content='\nAnother issue that has also h...,7,[0.951094388961792],False,chroma,tfidf
10,6,Project start date / end date,March 2023 – February 2026,"The project start date is March 2023, and the ...",[page_content='\n Identification and mapping ...,10,[0.25516653060913086],False,chroma,tfidf
12,7,Project budget \n Total amount of t...,The total required resources is 818 341 € for ...,- **Total amount of the project (in Euros):** ...,[page_content='\nMain Sustainable Development ...,7,[0.7097054123878479],False,chroma,tfidf



-----------

Dense type: chroma,
 Sparse type: tfidf,
 enhanced prompt: True
Score moyen llm eval: 7.714285714285714


,question_#,question,reference_answer,rag_answer,rag_sources,eval_score_llm,eval_score_minicheck,enhanced_question,dense_retriever_type,spare_retriever_type
1,1,What is the detailed description of the projec...,\n Brief project description\n ...,"The project, titled ""Mahakam,"" is focused on m...",[page_content='\n- Community members (villager...,7,[0.4817027747631073],True,chroma,tfidf
3,2,What is the specific country and city where th...,\n The location of the project is i...,The specific country where the project is loca...,[page_content='\n- Community members (villager...,7,[0.9235244393348694],True,chroma,tfidf
5,3,Who are the target beneficiaries of the project?,\n Number of direct beneficiaries o...,The target beneficiaries of the project includ...,[page_content='\nFeature 3. Theory of Change\n...,7,[0.20092874765396118],True,chroma,tfidf
7,4,How many individuals are impacted by the project?,\n Number of direct beneficiaries o...,"The project impacts a total of 6,245 individua...",[page_content='\nFeature 3. Theory of Change\n...,8,[0.18638822436332703],True,chroma,tfidf
9,5,"What are the contextual factors, environmental...",\n Context & environment and develo...,"The project in East Kalimantan, specifically i...",[page_content='\n- Community members (villager...,7,[0.6853238344192505],True,chroma,tfidf
11,6,What are the start and end dates of the project?,March 2023 – February 2026,The project is scheduled to start in March 202...,[page_content='\nMain Sustainable Development ...,10,[0.7088745832443237],True,chroma,tfidf
13,7,"What is the total project budget in Euros, inc...",The total required resources is 818 341 € for ...,"The total project budget is €818,341 for the d...",[page_content='\nMain Sustainable Development ...,8,[0.214956596493721],True,chroma,tfidf



-----------

Dense type: chroma,
 Sparse type: bm25,
 enhanced prompt: False
Score moyen llm eval: 7.142857142857143


...


-----------

Dense type: faiss,
 Sparse type: bm25,
 enhanced prompt: False
Score moyen llm eval: 7.714285714285714


,question_#,question,reference_answer,rag_answer,rag_sources,eval_score_llm,eval_score_minicheck,enhanced_question,dense_retriever_type,spare_retriever_type
42,1,Description of the project,\n Brief project description\n ...,"The project titled ""MAHAKAM"" is a mangrove res...",[page_content='\nHuman resources – Planète Urg...,6,[0.06616741418838501],False,faiss,bm25
44,2,Country and city,\n The location of the project is i...,"The country is Indonesia, and the relevant cit...",[page_content='MANGROVE RESTORATION AND\nLOCAL...,6,[0.03167888894677162],False,faiss,bm25
46,3,Target beneficiaries,\n Number of direct beneficiaries o...,The target beneficiaries of the pilot project ...,[page_content='\nFeature 3. Theory of Change\n...,8,[0.10996547341346741],False,faiss,bm25
48,4,Number of people concerned,\n Number of direct beneficiaries o...,The number of direct beneficiaries of the pilo...,[page_content='\nList of \nparticipants\nMoU D...,9,[0.16094541549682617],False,faiss,bm25
50,5,"Context, environment, project rationale and ch...",\n Context & environment and develo...,The context of the project revolves around the...,[page_content='\nAnother issue that has also h...,7,[0.9153727889060974],False,faiss,bm25
52,6,Project start date / end date,March 2023 – February 2026,"The project start date is March 2023, and the ...",[page_content='\n Identification and mapping ...,10,[0.25516653060913086],False,faiss,bm25
54,7,Project budget \n Total amount of t...,The total required resources is 818 341 € for ...,"The total amount of the project is €818,341. T...",[page_content='\nMain Sustainable Development ...,8,[0.31312671303749084],False,faiss,bm25



-----------

Dense type: faiss,
 Sparse type: bm25,
 enhanced prompt: True
Score moyen llm eval: 7.571428571428571


,question_#,question,reference_answer,rag_answer,rag_sources,eval_score_llm,eval_score_minicheck,enhanced_question,dense_retriever_type,spare_retriever_type
43,1,What is the detailed description of the project?,\n Brief project description\n ...,"The project, titled ""Mahakam,"" is focused on m...",[page_content='\n- Community members (villager...,8,[0.851862370967865],True,faiss,bm25
45,2,What is the specific country and city where th...,\n The location of the project is i...,"The project is located in Indonesia, specifica...",[page_content='MANGROVE RESTORATION AND\nLOCAL...,6,[0.9208370447158813],True,faiss,bm25
47,3,Who are the target beneficiaries of the project?,\n Number of direct beneficiaries o...,The target beneficiaries of the project includ...,[page_content='\nFeature 3. Theory of Change\n...,7,[0.18923136591911316],True,faiss,bm25
49,4,How many individuals are impacted by the project?,\n Number of direct beneficiaries o...,"The project impacts a total of 6,245 individua...",[page_content='\nFeature 3. Theory of Change\n...,8,[0.1740250438451767],True,faiss,bm25
51,5,"What are the contextual factors, environmental...",\n Context & environment and develo...,The project is situated in East Kalimantan Pro...,[page_content='\n- Community members (villager...,6,[0.16552533209323883],True,faiss,bm25
53,6,What are the start and end dates of the project?,March 2023 – February 2026,The project is scheduled to start in March 202...,[page_content='\nMain Sustainable Development ...,10,[0.7088745832443237],True,faiss,bm25
55,7,"What is the total project budget in Euros, inc...",The total required resources is 818 341 € for ...,"The total project budget is €818,341 for the d...",[page_content='\nMain Sustainable Development ...,8,[0.18756112456321716],True,faiss,bm25



-----------



In [26]:
df_evaluations_lecel[(df_evaluations_lecel["eval_score_llm"]<7)&(df_evaluations_lecel["enhanced_question"]==True)]

,question_#,question,reference_answer,rag_answer,rag_sources,eval_score_llm,eval_score_minicheck,enhanced_question,dense_retriever_type,spare_retriever_type
17,2,What is the specific country and city where th...,\n The location of the project is i...,The specific country where the project is loca...,[page_content='\n- Community members (villager...,6,[0.9493037462234497],True,chroma,bm25
45,2,What is the specific country and city where th...,\n The location of the project is i...,"The project is located in Indonesia, specifica...",[page_content='MANGROVE RESTORATION AND\nLOCAL...,6,[0.9208370447158813],True,faiss,bm25
51,5,"What are the contextual factors, environmental...",\n Context & environment and develo...,The project is situated in East Kalimantan Pro...,[page_content='\n- Community members (villager...,6,[0.16552533209323883],True,faiss,bm25


In [27]:
_best_params= pd.DataFrame(best_params).sort_values(by=3, ascending=False).head(1).values[0]
display(_best_params)
df_evaluations_lecel[
    (df_evaluations_lecel["dense_retriever_type"]==_best_params[0])&
    (df_evaluations_lecel["spare_retriever_type"]==_best_params[1])&
    (df_evaluations_lecel["enhanced_question"]==_best_params[2])
]#.to_csv("./rag-mahakam-hybrid-retriever-best-params.csv", index=False)


array(['faiss', 'tfidf', False, 7.857142857142857], dtype=object)

,question_#,question,reference_answer,rag_answer,rag_sources,eval_score_llm,eval_score_minicheck,enhanced_question,dense_retriever_type,spare_retriever_type
28,1,Description of the project,\n Brief project description\n ...,"The project titled ""Mahakam"" is focused on man...",[page_content='\nHuman resources – Planète Urg...,6,[0.05878777056932449],False,faiss,tfidf
30,2,Country and city,\n The location of the project is i...,"The country is Indonesia, and the relevant cit...",[page_content='MANGROVE RESTORATION AND\nLOCAL...,6,[0.018860161304473877],False,faiss,tfidf
32,3,Target beneficiaries,\n Number of direct beneficiaries o...,The target beneficiaries of the pilot project ...,[page_content='\nFeature 3. Theory of Change\n...,9,[0.1266552358865738],False,faiss,tfidf
34,4,Number of people concerned,\n Number of direct beneficiaries o...,The number of direct beneficiaries of the pilo...,[page_content='\nTable 2. Types of Land Cover ...,9,[0.15698452293872833],False,faiss,tfidf
36,5,"Context, environment, project rationale and ch...",\n Context & environment and develo...,The context outlines the significant impact of...,[page_content='\nAnother issue that has also h...,8,[0.8946554660797119],False,faiss,tfidf
38,6,Project start date / end date,March 2023 – February 2026,"The project start date is March 2023, and the ...",[page_content='\n Identification and mapping ...,10,[0.25516653060913086],False,faiss,tfidf
40,7,Project budget \n Total amount of t...,The total required resources is 818 341 € for ...,- **Total amount of the project (in Euros):** ...,[page_content='\nMain Sustainable Development ...,7,[0.06654772907495499],False,faiss,tfidf


In [28]:
display(pd.pivot_table(
    df_evaluations_lecel,#[df_evaluations_lecel["search_type"]=="mmr"],
    values="eval_score_llm",
    index="spare_retriever_type",
    columns="enhanced_question",
    aggfunc="mean" 
).round(2))

display(pd.pivot_table(
    df_evaluations_lecel,#[df_evaluations_lecel["search_type"]=="mmr"],
    values="eval_score_llm",
    index="dense_retriever_type",
    columns="enhanced_question",
    aggfunc="mean" 
).round(2))

enhanced_question,False,True
spare_retriever_type,,
bm25,7.43,7.57
tfidf,7.71,7.79


enhanced_question,False,True
dense_retriever_type,,
chroma,7.36,7.64
faiss,7.79,7.71


In [34]:
import textwrap
num_question=2
dense_retriever_type="chroma"
sparse_retriever_type="tfidf"

row=df_evaluations_lecel[(df_evaluations_lecel["enhanced_question"]==True)&
                            (df_evaluations_lecel["dense_retriever_type"]==dense_retriever_type)&
                            (df_evaluations_lecel["spare_retriever_type"]==sparse_retriever_type)&
                            (df_evaluations_lecel["question_#"]==num_question)]



fragments=row["rag_sources"]

print("Question:\n", row["question"].values[0])

print("\n------\nreference_answer:\n", row["reference_answer"].values[0])

print((f"\n---------\nrag_answer:\n{row['rag_answer'].values[0]}"))

print((f"\n---------\nRetrieved docs:\n{fragments.values[0]}"))


Question:
 What is the specific country and city where the project is located?

------
reference_answer:
 
            The location of the project is in Paser District (Adang Bay village) and Kutai
            Kartanegara district (Delta Mahakam) East Kalimantan Province. The location
            of project is nearby the new capital of Indonesia which is in the Penajam
            Paser Utara (around 130-160 km)
        

---------
rag_answer:
The project is located in Indonesia, specifically in the city of Samarinda, which is in East Kalimantan Province.

---------
Retrieved docs:
[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 19, 'score': 0.0023258572909981012}, page_content='\n- Community members (villagers, consists of fish farmers, women group,\nand youth)\n- Village officials\n- Stakeholders  from  various  institutions  (government  institutions,\nuniversities, and non-government organizations)\n- Public audience in general (reached

#### Conclusions:
* Un retriever hybride ne permet pas d'améliorer les résultats par rapport à un dense retriever
#### Prochaines étapes:
* Expérimenter avec du self rag